Code to test sigma coordinate conversion for ROMS output
Note: This uses special cases and is not necessarily generic like the function will be.

In [1]:
import numpy as np
import xarray as xr
from set_depth import set_depth

Access Data from the model

In [2]:
url='http://oceanmodeling.pmc.ucsc.edu:8080/thredds/dodsC/ccsra_2016a_phys_agg_slevs/fmrc/CCSRA_2016a_Phys_ROMS_Sigma-level_Aggregation_best.ncd'


In [3]:
model=xr.open_dataset(url)

Get S coordinate stretching curves at RHO-points

In [4]:
Cs_r=model['Cs_r'] # S-coordinate stretching curves at RHO-points
Cs_r=Cs_r.values

In [5]:
Cs_w=model['Cs_w'] # S-coordinate stretching curves at w-points
Cs_w=Cs_w.values

In [6]:
Vstretching=model['Vstretching'] # vertical terrain following stretching function

In [7]:
Vtransform=model['Vtransform'] # vertical terrain following tranformation equation

Get bathymetry and other model parameters need to compute depth from sigma coordinate

In [8]:
h=model['h'] # bathymetry at rho points

In [9]:
hc=model['hc'] # S-coordinate parameter, critical depth

In [10]:
theta_b=model['theta_b'] # S-coordinate bottom control parameter
theta_b=theta_b.values

In [11]:
theta_s=model['theta_s'] # S-coordinate surface control parameter
theta_s=theta_s.values

In [12]:
zeta=model['zeta'] # free surface

In [13]:
N=42 # number of levels, note w has 43 levels

In [14]:
igrid=1 # density point grid (T,S)
# use igrid=3 for u and igrid=4 for v
# use igrid=5 for w


In [15]:
report=0
zeta=np.squeeze(zeta[0,:,:])

In [16]:
z=set_depth(Vtransform,Vstretching,theta_s,theta_b,hc,N,igrid,h,zeta,report)

In [17]:
z.shape

(181, 186, 42)

In [18]:
igrid=2

In [19]:
zs=set_depth(Vtransform,Vstretching,theta_s,theta_b,hc,N,igrid,h,zeta,report)

In [20]:
zs.shape

(180, 185, 42)

In [21]:
#[Lp,Mp]=h.shape
#L=Lp-1
#M=Mp-1
##hu=0.5*(h[0:L,0:Mp]+h[1:Lp,0:Mp])
#hu=0.5*(h[0:Lp,0:M]+h[0:Lp,1:Mp])
#hu.shape

In [22]:
igrid=3
zu=set_depth(Vtransform,Vstretching,theta_s,theta_b,hc,N,igrid,h,zeta,report)
zu.shape

(181, 185, 42)

In [23]:
igrid=4
zv=set_depth(Vtransform,Vstretching,theta_s,theta_b,hc,N,igrid,h,zeta,report)
zv.shape

(180, 186, 42)

In [24]:
#model.transpose(('eta_rho','xi_rho','s_rho','time'))
temp=model['temp']
salt=model['salt']
u=model['u']
v=model['v']
print(temp.shape)
print(u.shape)
print(v.shape)

(3212, 42, 181, 186)
(3212, 42, 181, 185)
(3212, 42, 180, 186)


Grab an actual time 

In [25]:
atemp=temp[0,:,:,:]
asalt=salt[0,:,:,:]

Get the actual values so we can rearrange to an array we want

In [26]:
atemp=atemp.values
asalt=asalt.values

Transpose to the shape we want so z is the third axis
This will now have the shape of as the z data

In [27]:
atemp=atemp.transpose((1,2,0))
asalt=asalt.transpose((1,2,0))

In [28]:
print(atemp.shape)
print(z.shape)

(181, 186, 42)
(181, 186, 42)


Need to find points within Sanctuary Boundary

In [29]:
from shapely.geometry import Polygon, Point
import scipy.io

Below is for MPA not Sanctuary

In [30]:
file='c:\mbns_file\mbns_outline.mat'
mbns=scipy.io.loadmat(file)
lat=mbns['lat']
long=mbns['long']
lat=np.transpose(lat)
long=np.transpose(long)
#file='c:\MPA\ds582\ds582.shp'

In [31]:
lat.shape

(6666, 1)

get the model grid data

In [32]:
modellat=model['lat_rho']
modellon=model['lon_rho']

Read the shapefile for the sanctuary

In [33]:
from matplotlib.path import Path
apoly=[]
lat=lat.flatten()
long=long.flatten()
lat=np.array(lat)
long=np.array(long)
for i in range(0,6666):
    apoly.append((long[i],lat[i]))


In [34]:
p=Path(apoly)

flatten grid for comparison

In [35]:
flat=modellat.values.flatten()
flon=modellon.values.flatten()

In [36]:
points=np.vstack((flon,flat)).T

Now find the model points within the sanctuary

In [37]:
thegrid=p.contains_points(points)

Okay now we have the points in the santuary...
What is next? compute heat content for those points?

In [38]:
sanctuarylats=flat[thegrid]
sanctuarylons=flon[thegrid]

Okay the following works but seems counter to what it should be.  
mla is the same as sanctuarylats above

In [39]:
nx=np.arange(0,181)
ny=np.arange(0,186)
[mx,my]=np.meshgrid(ny,nx)

In [40]:
mx=mx.flatten()
my=my.flatten()

In [41]:
xindex=mx[thegrid]
yindex=my[thegrid]

In [42]:
mla=modellat.values
mlo=modellon.values

In [43]:
mla=mla[yindex,xindex]
mlo=mlo[yindex,xindex]

In [44]:
ztest=z[yindex,xindex,:]
ttest=atemp[yindex,xindex,:]
stest=asalt[yindex,xindex,:]

In [45]:
ztest.shape

(140, 42)

Okay this is a flattened version of the z array I think... 
Need to compute heat capacity and then the heat of the volume of water?

In [47]:
import seawater as sw

In [48]:
heatcapacity=sw.cp(stest,ttest,ztest) # compute heat capacity of seawater

In [49]:
density=sw.dens(stest,ttest,ztest) # compute density of seawhater

In [ ]:
#So heat content is dens*cp*int(T(z)dz) from h1 to h2
#Let's loop through the points in the sanctuary and integrate vertically

In [81]:
[num,nz]=ztest.shape
for j in np.arange(0,num):
    sum=0
    for i in np.arange(0,nz-1):
        # surface area is 10*10 km or 100 km^2 (ignoring the coastline)
        if ztest[j,i] < 100:
            dz=ztest[j,i+1]-ztest[j,i]
            dT=ttest[j,i+1]-ttest[j,i]
            triang=0.5*dz*dT
            partial=heatcapacity[j,i]*density[j,i]*(ttest[j,i]*dz+triang)
            sum=sum+partial
    if j==0:
        heat=sum
    else:
        heat=np.append(heat,sum)

In [82]:
# heat has units of J/m^2 so if we multiply by 100*(1000^2)
#heat=heat*100*1000*1000 # now we are in units of Joules
heat=heat/1e+9 # convert to GigaJoules

In [83]:
heat

array([12.70698489,  9.59390309,  7.01354478,  5.04737063,  3.58729465,
       13.47262844, 10.2533318 ,  7.66785756,  5.53551142,  0.        ,
       11.1183334 ,  8.18233019,  5.87198946,  4.24426557,  8.93440891,
        6.39265321,  4.46743869, 17.53963695, 13.47288176, 10.05277128,
        7.13121052,  4.88141791, 28.49358143, 23.9003121 , 19.29747418,
       14.83486619, 10.90554504,  7.89580717,  5.43601022, 29.83225482,
       25.36098988, 20.83442196, 16.5834882 , 12.53720003,  8.85362833,
        6.03190359, 31.795685  , 27.43139453, 22.72320876, 18.14497701,
       14.036845  , 10.36212229,  6.9978487 , 34.16900272, 30.15274688,
       25.7758637 , 20.87583258, 16.0720935 , 11.90796524,  8.47897426,
       35.37426463, 31.89212294, 27.81997281, 23.53770768, 18.82012191,
       14.1602413 , 10.25007426,  7.09193741, 32.41934968, 28.75385401,
       24.53785174, 20.21362891, 15.86195384, 11.8212935 ,  8.63166388,
        6.08579582, 31.71511816, 28.49005578, 24.73613604, 20.52